# Lookup API Integrations 

This notebook gives examples of how to use these API integrations.

## Loading Test Data

In [1]:
from objects import get_ioc_type

iocs:list[str] = open('data/test-input-iocs.txt', 'r').read().splitlines()
print(f'\033[90mGot \033[92m{len(iocs)} \033[90mtotal IOCs.')

# Convert the iocs to dicts with the value and type
iocs_as_dicts:list[dict] = []
for ioc in iocs: 
    iocs_as_dicts.append({
        'value': ioc,
        'type': get_ioc_type(ioc)
    })


Got 100 total IOCs.


## Loading config

In [2]:
import json 

with open('config/config.json', 'r') as file: 
    config:dict = json.load(file)               # Loading the config json
    tokens:dict[str, str] = config['tokens']    # Get the API tokens from the config dict
    
# Defining an output directory for analyzing results of testing/understanding output formats 
output_dir:str = 'data/test-outputs/' 

## Init APIs

In [3]:
from objects import IPInfoAPI, VirusTotalAPI

ipinfo_api:IPInfoAPI = IPInfoAPI(tokens['ipinfo'])
virustotal_api:VirusTotalAPI = VirusTotalAPI(tokens['virustotal'])

## Conduct lookups on all the IOCs using each API

In [4]:
vt_results:list[dict] = virustotal_api.lookup_iocs(iocs_as_dicts)
ipinfo_results:list[dict] = ipinfo_api.lookup_iocs(iocs_as_dicts)

NOTICE: No results found for "63.116.104.5" (VirusTotal)
NOTICE: No results found for "104.238.130.180" (VirusTotal)
NOTICE: No results found for "149.56.1.33" (VirusTotal)
NOTICE: No results found for "173.194.0.56" (VirusTotal)
NOTICE: No results found for "185.60.216.35" (VirusTotal)
NOTICE: No results found for "204.79.197.200" (VirusTotal)
NOTICE: No results found for "205.251.242.103" (VirusTotal)
NOTICE: No results found for "13.35.20.0" (VirusTotal)
NOTICE: No results found for "23.211.123.0" (VirusTotal)
NOTICE: No results found for "34.117.59.0" (VirusTotal)
NOTICE: No results found for "google.com" (VirusTotal)
NOTICE: No results found for "facebook.com" (VirusTotal)
NOTICE: No results found for "amazon.com" (VirusTotal)
NOTICE: No results found for "apple.com" (VirusTotal)
NOTICE: No results found for "microsoft.com" (VirusTotal)
NOTICE: No results found for "yahoo.com" (VirusTotal)
NOTICE: No results found for "youtube.com" (VirusTotal)
NOTICE: No results found for "wikipe

**Combine the VT and IPInfo results**

In [5]:
from objects import combine_vt_ipinfo_results, SSLCert
import pandas as pd

# Combine the results
ip_results, domain_results, sslcert_results = combine_vt_ipinfo_results(
    { ioc['value'] : ioc for ioc in vt_results },
    { ioc['value'] : ioc for ioc in ipinfo_results }
)

# Create dataframes from the results
ip_results_df:pd.DataFrame = pd.DataFrame(ip_results)
domain_results_df:pd.DataFrame = pd.DataFrame(domain_results)
sslcert_results_df:pd.DataFrame = pd.DataFrame([ SSLCert(d).to_table_rows() for d in sslcert_results ])


**Dump the combined results to JSONs and CSV/Excel**

In [6]:
with open(output_dir + 'ip_results.json', 'w+') as file: 
    json.dump(ip_results, file, indent=4)
    
with open(output_dir + 'domain_results.json', 'w+') as file: 
    json.dump(domain_results, file, indent=4)
    
with open(output_dir + 'sslcert_results.json', 'w+') as file: 
    json.dump(sslcert_results, file, indent=4)
    
ip_results_df.to_csv(output_dir + 'ip_results.csv', index=False)
domain_results_df.to_csv(output_dir + 'domain_results.csv', index=False)
sslcert_results_df.to_csv(output_dir + 'sslcert_results.csv', index=False)

In [7]:
with open('vt_results.json', 'w+') as file:
    json.dump(vt_results, file, indent=4)